In [1]:
import os
from langchain_community.document_loaders import TextLoader
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
loader = TextLoader("./data/speech.txt")
print(loader)

In [4]:
text_document = loader.load()
text_document

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document_chunks = text_splitter.split_documents(text_document)
print(len(document_chunks))
print(document_chunks[0])

5
page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.

…' metadata={'source': 'speech.txt'}


In [11]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [13]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(document_chunks, embeddings)
vectorstore

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following questions based only on the provided context:
    <context>
    {context}
    </context>
    """
)

In [15]:
llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x118e72330> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x118e73ec0> root_client=<openai.OpenAI object at 0x11e007410> root_async_client=<openai.AsyncOpenAI object at 0x118e709b0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x118e72330>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x118e73ec0>, root_client=<openai.OpenAI object at 0x11e007410>, root_async_client=<openai.AsyncOpenAI object at 0x118e709b0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_nam

In [18]:
retriever = vectorstore.as_retriever()

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11e004560>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
    

In [21]:
result = retrieval_chain.invoke({'input':"Provide summary of the speech?"})
result

{'input': 'Provide summary of the speech?',
 'context': [Document(id='172fb603-1697-4a8e-b6d1-e98e6b7edf3b', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  Document(id='d0a74ade-713f-49a7-8659-73c60031d6

In [23]:
print(result['answer'])

1. **What is the speaker's main message to the Congress?**

   The speaker's main message to Congress is to justify entering into war, not out of selfish intentions or desires for conquest, but to protect and uphold precious ideals such as democracy, the rights of small nations, and universal justice. The speaker emphasizes the fight for democracy and political liberty, and the aim to make the world safe for these concepts, insisting that while the war is daunting, it is necessary to protect and secure the rights of mankind.

2. **How does the speaker propose the nation should conduct itself in the war?**

   The speaker proposes that the nation should conduct itself in the war with high standards of fairness and justice, without rancor or selfish ambitions. The nation should act as a champion of the rights of mankind, fighting for principles that can be shared with all free peoples. Moreover, operations as belligerents should be conducted without passion, observing the principles of r